# Titanic - XGBoost

In [1]:
# Version Check
from platform import python_version
python_version()

'3.7.6'

Importing all necessary packages

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
sns.set()
%matplotlib inline

### Task 1: Load Data from 'titanic_clean.csv'

In [3]:
#write code here
data = pd.read_csv('datasets/titanic_clean.csv')
df = data.copy()
df.head()

,PassengerId,Survived,Pclass,Sex,SibSp,Parch,Embarked,Title,GrpSize,FareCat,AgeCat
0,1,0,3,male,1,0,S,Mr,couple,0-10,16-32
1,2,1,1,female,1,0,C,Mrs,couple,70-100,32-48
2,3,1,3,female,0,0,S,Miss,solo,0-10,16-32
3,4,1,1,female,1,0,S,Mrs,couple,40-70,32-48
4,138,0,1,male,1,0,S,Mr,couple,40-70,32-48


In [4]:
print(df.shape)

(891, 11)


### One Hot encoding for categorical varaibles

In [5]:
df_OneHot=pd.get_dummies(df,columns=['Pclass','Sex','Embarked','Title','GrpSize','FareCat','AgeCat'])
df_OneHot.head()

,PassengerId,Survived,SibSp,Parch,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,...,FareCat_100+,FareCat_25-40,FareCat_40-70,FareCat_70-100,FareCat_Oct-25,AgeCat_0-16,AgeCat_16-32,AgeCat_32-48,AgeCat_48-64,AgeCat_64+
0,1,0,1,0,0,0,1,0,1,0,...,0,0,0,0,0,0,1,0,0,0
1,2,1,1,0,1,0,0,1,0,1,...,0,0,0,1,0,0,0,1,0,0
2,3,1,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,4,1,1,0,1,0,0,1,0,0,...,0,0,1,0,0,0,0,1,0,0
4,138,0,1,0,1,0,0,0,1,0,...,0,0,1,0,0,0,0,1,0,0


In [6]:
df=df_OneHot.copy()

### Task 2: Create Independent and Dependent Variables

In [7]:
#write code here
X = df.drop(columns= ['Survived','PassengerId'], axis =1)
Y = df['Survived']

### Task 3: Train Test n Split the data

In [8]:
# Import the library
from sklearn.model_selection import train_test_split

In [9]:
#Write the code here
xtrain, xtest, ytrain, ytest = train_test_split(X,Y,test_size=0.3, random_state=25, shuffle = True)
print(xtrain.shape, ytrain.shape)
print(xtest.shape, ytest.shape)

(623, 30) (623,)
(268, 30) (268,)


In [10]:
xtrain.head()

,SibSp,Parch,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,...,FareCat_100+,FareCat_25-40,FareCat_40-70,FareCat_70-100,FareCat_Oct-25,AgeCat_0-16,AgeCat_16-32,AgeCat_32-48,AgeCat_48-64,AgeCat_64+
707,0,0,0,0,1,0,1,0,0,1,...,0,0,0,0,1,0,1,0,0,0
246,0,1,0,0,1,0,1,0,0,1,...,0,0,0,0,1,0,0,1,0,0
624,1,1,1,0,0,1,0,0,0,1,...,0,0,0,1,0,0,0,0,1,0
96,0,0,0,1,0,1,0,0,0,1,...,0,0,0,0,1,0,1,0,0,0
576,0,0,0,0,1,0,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [11]:
xtrain.columns

Index(['SibSp', 'Parch', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female',
       'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S', 'Title_Master',
       'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Rare Title',
       'GrpSize_couple', 'GrpSize_group', 'GrpSize_large group',
       'GrpSize_solo', 'FareCat_0-10', 'FareCat_100+', 'FareCat_25-40',
       'FareCat_40-70', 'FareCat_70-100', 'FareCat_Oct-25', 'AgeCat_0-16',
       'AgeCat_16-32', 'AgeCat_32-48', 'AgeCat_48-64', 'AgeCat_64+'],
      dtype='object')

In [12]:
xtrain['FareCat_10-25'] = xtrain.loc[:,'FareCat_Oct-25'].copy()
xtrain.drop(columns='FareCat_Oct-25',inplace=True)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
D:\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [13]:
xtest['FareCat_10-25'] = xtest.loc[:,'FareCat_Oct-25'].copy()
xtest.drop(columns='FareCat_Oct-25',inplace=True)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
ytrain.head()

707    0
246    0
624    1
96     1
576    0
Name: Survived, dtype: int64

# XGBoost

### Creating Model & Training

In [15]:
from sklearn.model_selection import RandomizedSearchCV

In [16]:
min_child_weight = [int(x) for x in np.linspace(start=1, stop=3, num=3)]

n_estimators = [int(x) for x in np.linspace(start=1, stop=10, num= 3)]

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(1, 5, num=3)]
#max_depth.append(None)

eta = [ 0.5, 0.6, 0.7]

gamma = [ 0.8, 0.9, 1.0]

colsample_bytree = [0.7, 0.8, 0.9]


In [17]:
parameters = {
    'eta'              : eta, 
    'max_depth'        : max_depth,  
    #'objective'        : 'multi:softprob',
    "min_child_weight" : min_child_weight,
    "gamma"            : gamma,
    "colsample_bytree" : colsample_bytree,
    "n_estimators"     : n_estimators}
    #'num_class'        : 2} 


'''
The simplest parameters are the max_depth (maximum depth of the decision trees being trained),
objective (the loss function being used), and num_class (the number of classes in the dataset).

The eta parameter gives us a chance to prevent this overfitting.
The eta can be thought of more intuitively as a learning rate. 
Rather than simply adding the predictions of new trees to the ensemble with full weight,
the eta will be multiplied by the residuals being adding to reduce their weight.

This effectively reduces the complexity of the overall model.
It is common to have small values in the range of 0.1 to 0.3. 
The smaller weighting of these residuals will still help us train a powerful model, 
but won’t let that model run away into deep complexity where overfitting is more likely to happen.

The gamma parameter can also help with controlling overfitting. 
It specifies the minimum reduction in the loss required to make a further partition on a leaf node of the tree. 
I.e if creating a new node doesn’t reduce the loss by a certain amount, then we won’t create it at all.

The booster parameter allows you to set the type of model you will use when building the ensemble. 
The default is gbtree which builds an ensemble of decision trees. If your data isn’t too complicated, 
you can go with the faster and simpler gblinear option which builds an ensemble of linear models.
'''

steps = 500  # The number of training iterations

In [18]:
clf = xgb.XGBClassifier()
xgb_random = RandomizedSearchCV(
                    estimator=clf,
                    param_distributions= parameters,
                    n_jobs=4,
                    n_iter=1000,
                    scoring="neg_log_loss",
                    cv=5,
                    verbose=1,
                    random_state=100)
# Fit the random search model
xgb_random.fit(xtrain, ytrain)

D:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:282: UserWarning: The total space of parameters 729 is smaller than n_iter=1000. Running 729 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 729 candidates, totalling 3645 fits


[Parallel(n_jobs=4)]: Done  56 tasks      | elapsed:    3.5s
[Parallel(n_jobs=4)]: Done 1232 tasks      | elapsed:   12.6s
[Parallel(n_jobs=4)]: Done 3232 tasks      | elapsed:   29.1s
[Parallel(n_jobs=4)]: Done 3645 out of 3645 | elapsed:   33.2s finished


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100,...
                                           reg_lambda=None,
                                           scale_pos_weight=None,
                                           subsample=No

In [19]:
clf_xgb_rand = xgb_random.best_estimator_
clf_xgb_rand.fit(xtrain, ytrain)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, eta=0.6, gamma=0.9,
              gpu_id=-1, importance_type='gain', interaction_constraints=None,
              learning_rate=0.600000024, max_delta_step=0, max_depth=3,
              min_child_weight=2, missing=nan, monotone_constraints=None,
              n_estimators=5, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method=None, validate_parameters=False, verbosity=None)

In [20]:
pred_xgb_rand = clf_xgb_rand.predict(xtest)


First value in Numpy array is probability of **0** and second is probability of **1**

Only predicting and extracting probability values of **1**

In [21]:
prb_xgb_rand = clf_xgb_rand.predict_proba(xtest)[:, 1] #probabilities of 1 (survival)

In [22]:
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix

In [23]:
xgb_random_recall= recall_score(ytest,pred_xgb_rand)
xgb_random_acc= accuracy_score(ytest,pred_xgb_rand)
xgb_random_pre= precision_score(ytest,pred_xgb_rand)
xgb_random_f1_score= f1_score(ytest,pred_xgb_rand)

print("XGBoost Hyperparameters Tuning Using Randomized Search")
print("Precision: {0:.3f}%".format(xgb_random_pre*100))
print("Accuracy:  {0:.3f}%".format(xgb_random_acc*100))
print("Recall:    {0:.3f}%".format(xgb_random_recall*100))
print("F1-Score:  {0:.3f}%".format(xgb_random_f1_score*100))

XGBoost Hyperparameters Tuning Using Randomized Search
Precision: 84.000%
Accuracy:  82.090%
Recall:    63.636%
F1-Score:  72.414%


In [24]:
tn, fp, fn, tp = confusion_matrix(ytest, pred_xgb_rand).ravel()
conf_matrix=pd.DataFrame({"pred_Survived":[tp,fp],"pred_Not Survived":[fn,tn]},index=["Survived","Not Survived"])
conf_matrix

,pred_Survived,pred_Not Survived
Survived,63,36
Not Survived,12,157


## 3. Submission on Kaggle

### Task 4: Import test data

In [25]:
#write code here
test = pd.read_csv('competition_clean.csv')
df_test = test.copy()
df_test.head()


,PassengerId,Pclass,Sex,SibSp,Parch,Embarked,Title,GrpSize,FareCat,AgeCat
0,892,3,male,0,0,Q,Mr,solo,0-10,32-48
1,893,3,female,1,0,S,Mrs,couple,0-10,32-48
2,894,2,male,0,0,Q,Mr,solo,0-10,48-64
3,895,3,male,0,0,S,Mr,solo,0-10,16-32
4,896,3,female,1,1,S,Mrs,group,10-25,16-32


### Task 5: Do One Hot encoding of test data

In [26]:
#write code here
df_OneHot=pd.get_dummies(df_test,columns=['Pclass','Sex','Embarked','Title','GrpSize','FareCat','AgeCat'])
df_OneHot.head()

,PassengerId,SibSp,Parch,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,...,FareCat_10-25,FareCat_100+,FareCat_25-40,FareCat_40-70,FareCat_70-100,AgeCat_0-16,AgeCat_16-32,AgeCat_32-48,AgeCat_48-64,AgeCat_64+
0,892,0,0,0,0,1,0,1,0,1,...,0,0,0,0,0,0,0,1,0,0
1,893,1,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,894,0,0,0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
3,895,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
4,896,1,1,0,0,1,1,0,0,0,...,1,0,0,0,0,0,1,0,0,0


### Task 6: Separate Passenger ID for submission

In [27]:
#Write code here
PassengerID = df_OneHot['PassengerId']
xinput = df_OneHot.drop(columns = 'PassengerId', axis=1)
xinput

,SibSp,Parch,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,...,FareCat_10-25,FareCat_100+,FareCat_25-40,FareCat_40-70,FareCat_70-100,AgeCat_0-16,AgeCat_16-32,AgeCat_32-48,AgeCat_48-64,AgeCat_64+
0,0,0,0,0,1,0,1,0,1,0,...,0,0,0,0,0,0,0,1,0,0
1,1,0,0,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,1,0,1,0,0,1,...,0,0,0,0,0,0,1,0,0,0
4,1,1,0,0,1,1,0,0,0,1,...,1,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,0,0,0,0,1,0,1,0,0,1,...,0,0,0,0,0,0,1,0,0,0
414,0,0,1,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,1,0,0
415,0,0,0,0,1,0,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
416,0,0,0,0,1,0,1,0,0,1,...,0,0,0,0,0,0,1,0,0,0


### Task 7: Do prediction through final model

In [28]:
xin= xinput[['SibSp', 'Parch', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S', 'Title_Master', 'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Rare Title', 'GrpSize_couple', 'GrpSize_group', 'GrpSize_large group', 'GrpSize_solo', 'FareCat_0-10', 'FareCat_100+', 'FareCat_25-40', 'FareCat_40-70', 'FareCat_70-100', 'AgeCat_0-16', 'AgeCat_16-32', 'AgeCat_32-48', 'AgeCat_48-64', 'AgeCat_64+', 'FareCat_10-25']] 

In [29]:
#write code here
pred_final= clf_xgb_rand.predict(xin)
pred_final.shape

(418,)

#### Creating Data Frame for submission

In [30]:
submission=pd.DataFrame({'PassengerId':PassengerID,'Survived':pred_final})

In [31]:
submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [32]:
submission.set_index(submission.columns[0],inplace= True)

### Task 8: Export the dataset into csv file

In [33]:
# Write code here
submission.to_csv('Submission_XGBoost_ver_4.csv')